In [ ]:
import json
from nltk.tokenize.treebank import TreebankWordDetokenizer
with open('/content/train.json') as o:
        data = json.load(o)

with open('/content/tables.json') as o:
        Tables = json.load(o)

newNumricDBs = []
for db in Tables:
    # create list of col_index that are numric type
    numberIndexes = []
    i = 0
    for t in db['column_types']:
      if t == "number":
        numberIndexes.append(i)
      i = i + 1
    #print("numberIndexes. ", numberIndexes)
    # create list of columns names with table id that are numric type
    columnIndex = 0
    numricColList = []
    for c in db['column_names_original']:
      if columnIndex in numberIndexes:
        numricColList.append(c)
      columnIndex = columnIndex + 1
    #print("numricColList. ", numricColList)
    
    # create table indexes
    
    tableindex = 0
    tableindexList = []
    for t in db['table_names_original']:
      tableindexList.append([tableindex, t])
      tableindex = tableindex + 1
    
    for c in numricColList:
       for t in tableindexList:
         if c[0] == t[0]:
            c[0] = t[1]
    newNumricDBs.append([db['db_id'],numricColList])

         

In [ ]:
for db in newNumricDBs:
  print("before ",db)
  listindex = 0
  removeList = []
  for l in db[1]:
    print(l)
    if 'id' in l[1]:
      removeList.append(l)
    elif 'Id' in l[1]:
      removeList.append(l)
    elif 'ID' in l[1]:
      removeList.append(l)
    listindex = listindex + 1

  for item in removeList:
    db[1].remove(item)

  print("after ",db)

removeList = []
for db in newNumricDBs:
  if len(db[1]) == 0:
    removeList.append(db)
for db in removeList:
  newNumricDBs.remove(db)


In [ ]:
import random

questionStarter = ['show me the', 'what is the', 'calculate the', 'find']
Dist = ['normal distribution', 'standardized normal distribution', 'long tailed distribution', 'student t distribution', 'f distribution', 'bionomial distribution', 'poisson distribution', 'exponential distribution', 'weibull distribution', 'chi square distribution']
DistORG = ['normal', 'standard_normal', 'long_tailed', 't', 'f', 'binomial', 'poisson', 'exponential', 'weibull', 'chi_square']
transword1 = ['for the', 'for']
df = ['with degree of freedom of', 'with df equal', 'where df is']
c = ['where shape factor equal to', 'where c equal']
dfandC_values = [[1,"1"],[2,"2"],[3,"3"]]
ndf = [[1,"1"],[2,"2"],[3,"3"]]
ddf = [[13,"1"],[14,"2"],[15,"3"]]


newDistList = []
distindex = -1
for d in Dist:
  distindex = distindex + 1
  limit = 0
  for db in newNumricDBs:
    limit = limit + 1
    if limit > 30:
      break
    question = []
    question.append(random.choice(questionStarter))
    question.append(d)
    question.append(random.choice(transword1))
    tANDc = random.choice(db[1])
    question.append(tANDc[1])
    

    query = []
    query.append('distribution')
    query.append([DistORG[distindex]])
    query.append(tANDc[0])
    query.append([[tANDc[1]]])
    
    if DistORG[distindex] == 'weibull':
      question.append(random.choice(c))
      value = random.choice(dfandC_values) 
      question.append(value[1])
      query.append([['c','=',value[0]]])

    elif DistORG[distindex] == 't' or DistORG[distindex] == 'chi_square':
      question.append(random.choice(df))
      value = random.choice(dfandC_values) 
      question.append(value[1])
      query.append([['df','=',value[0]]])

    elif DistORG[distindex] == 'f':
      value1 = random.choice(ndf) 
      question.append("with nominator equal")
      question.append(value1[1])
      value2 = random.choice(ddf) 
      question.append("and denominator")
      question.append(value2[1])
      query.append([['ndf','=',value1[0],'ddf','=',value2[0]]])
    else:
      query.append([[]])

    
    sample = {}
    sample['db_id'] = db[0]
    question = TreebankWordDetokenizer().detokenize(question)
    sample['question'] = question
    sample['function'] = query
    newDistList.append(sample)



In [ ]:
with open('Dist_examples.json', 'wt') as data:
   json.dump(newDistList, data, sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
for e in newDistList:
  print(e)

In [ ]:
import random

questionStarter = ['show me the', 'Give the','Give me a', 'provide the','show a']
plot = ['histogram plot', 'box plot', 'desnity plot','pie chart', 'bar chart','scatter plot' ,'hexbin chart', 'contour figure', 'violin plot']
plotORG = ['hist', 'box', 'kde', 'pie', 'bar','scatter', 'hex', 'contour', 'violin']
transword1 = ['for the', 'for']



newPlotList = []
plotindex = -1
for p in plot:
  plotindex = plotindex + 1
  limit = 0
  for db in newNumricDBs:
    limit = limit + 1
    if limit > 30:
      break
    question = []
    question.append(random.choice(questionStarter))
    question.append(p)
    question.append(random.choice(transword1))
    tANDc1 = random.choice(db[1])    
    question.append(tANDc1[1])
    if plotindex == 3 or plotindex == 4 or plotindex == 5 or plotindex == 6 or plotindex == 7:
      count = 0
      tANDc2 = random.choice(db[1])
      while count < 1000 and tANDc2[0] != tANDc1[0]:
        tANDc2 = random.choice(db[1])
        count = count + 1 
      question.append("and")
      question.append(tANDc2[1])
    

    query = []
    query.append('plot')
    query.append([plotORG[plotindex]])
    query.append(tANDc1[0])
    query.append([[tANDc1[1]]])
    if plotindex == 3 or plotindex == 4 or plotindex == 5 or plotindex == 6 or plotindex == 7:
      query[3][0].append(tANDc2[1])
    query.append([[]])

    
    sample = {}
    sample['db_id'] = db[0]
    question = TreebankWordDetokenizer().detokenize(question)
    sample['question'] = question
    sample['function'] = query
    newPlotList.append(sample)


In [ ]:
with open('plot_examples.json', 'wt') as data:
   json.dump(newPlotList, data, sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
for e in newPlotList:
  print(e)

In [ ]:
import random

questionStarter = ['Calculate', 'What is the','Show me the','Find', 'Provide the','List the']
numric = ['mean', 'weighted mean', 'trimmed mean', 'mean absolute deviation', 'median'
          ,'weighted median','median absolute deviation','outlier','standard deviation','variance',
          'range','interquartile range','frequency table','mode','standard error',
          'percentile','correlation matrix','correlation coefficient','contingency table','size',
          'confidence interval']
numricORG = ['mean', 'w_mean', 't_mean', 'mean_ad', 'median'
          ,'w_median','median_ad','outlier','std','var',
          'range','iqr','frequency_table','mode','standard_error',
          'percentile','corr_matrix','corr_coff','contingency_table','size',
          'ci']
transword1 = ['for the', 'for','of the']
trimmedWords = ['with truncated percental of', 'with trimmed percental equal to']
percentWords = ['with percental equal to','where percental equal to']
positionPercental = ['in the percental position of']
percentValues = [[.1,'10%'],[.2,'20%'],[.3,'30%']]
ciValues = [[.90,'90%'],[.95,'95%'],[.97,'97%']]



newNumricList = []
numricindex = -1
for n in numric:
  numricindex = numricindex + 1
  limit = 0
  for db in newNumricDBs:
    limit = limit + 1
    if limit > 10:
      break
    question = []
    question.append(random.choice(questionStarter))
    question.append(n)
    question.append(random.choice(transword1))
    tANDc1 = random.choice(db[1]) 
    question.append(tANDc1[0])   
    question.append(tANDc1[1])

    if numricindex == 1 or numricindex == 5 or numricindex ==  12 or numricindex == 16 or numricindex == 17 or numricindex == 18:
      count = 0
      tANDc2 = random.choice(db[1])
      # handle two and three columns
      while count < 1000 and tANDc2[0] != tANDc1[0]:
        tANDc2 = random.choice(db[1])
        count = count + 1        
      if numricindex == 1 or numricindex == 5 or numricindex == 17:
          question.append("and")
          question.append(tANDc2[1])
      else:
          question.append(",")
          question.append(tANDc2[1])
      # handle three columns
      if numricindex == 12 or numricindex == 16 or numricindex ==  18:
          count = 0
          tANDc3 = random.choice(db[1])
          while count < 1000 and tANDc3[0] != tANDc1[0]:
             tANDc3 = random.choice(db[1])
             count = count + 1 
          question.append("and")
          question.append(tANDc3[1])
    
    if numricindex == 2:
      question.append(random.choice(trimmedWords))
      trimmedValue = random.choice(percentValues)
      question.append(trimmedValue[1])

    if numricindex == 15:
      question.append(random.choice(positionPercental))
      percentValue = random.choice(percentValues)
      question.append(percentValue[1])

    if numricindex == 20:
      question.append(random.choice(percentWords))
      ciValue = random.choice(ciValues)
      question.append(ciValue[1])


    

    query = []
    query.append('numric')
    query.append([numricORG[numricindex]])
    query.append(tANDc1[0])
    query.append([[tANDc1[1]]])
    if numricindex == 1 or numricindex == 5 or numricindex ==  12 or numricindex == 16 or numricindex == 17 or numricindex == 18:
      query[3][0].append(tANDc2[1])
    if numricindex == 12 or numricindex == 16 or numricindex ==  18:
      query[3][0].append(tANDc3[1])

    if numricindex == 2:
      query.append([['percent','=',trimmedValue[0]]])
    elif numricindex == 15:
      query.append([['percent','=',percentValue[0]]])
    elif numricindex ==  20:
      query.append([['percent','=',ciValue[0]]])
    else:
      query.append([[]])

    
    sample = {}
    sample['db_id'] = db[0]
    question = TreebankWordDetokenizer().detokenize(question)
    sample['question'] = question
    sample['function'] = query
    newNumricList.append(sample)


In [ ]:
with open('one_col_numric_examples.json', 'wt') as data:
   json.dump(newNumricList, data, sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
for e in newNumricList:
  print(e)

In [ ]:
import random

questionStarter = ['Calculate', 'What is the','Show me the','Find', 'Provide the','List the']
numric = ['mean', 'mean absolute deviation', 'median'
          ,'median absolute deviation','standard deviation','variance',
          'range','interquartile range','mode',
          'size']
numricORG = ['mean','mean_ad', 'median'
          ,'median_ad','std','var',
          'range','iqr','mode',
          'size']
transword1 = ['for the', 'for','of the']



newNumricList = []
numricindex = -1
for n in numricORG:
  numricindex = numricindex + 1
  limit = 0
  for db in newNumricDBs:
    limit = limit + 1
    if limit > 5:
      break
    question = []
    question.append(random.choice(questionStarter))
    question.append(n)
    question.append("and")
    secondN = random.choice(numricORG)
    question.append(secondN)
    question.append(random.choice(transword1))
    tANDc1 = random.choice(db[1]) 
    question.append(tANDc1[0])   
    question.append(tANDc1[1])


    query = []
    query.append('numric')
    query.append([n])
    query[1].append(secondN)
    query.append(tANDc1[0])
    query.append([[tANDc1[1]]])
    query[3].append([tANDc1[1]])

    query.append([[]])
    query[4].append([])

    
    sample = {}
    sample['db_id'] = db[0]
    question = TreebankWordDetokenizer().detokenize(question)
    sample['question'] = question
    sample['function'] = query
    newNumricList.append(sample)


In [ ]:
with open('two_col_numric_examples.json', 'wt') as data:
   json.dump(newNumricList, data, sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
for e in newNumricList:
  print(e)

{'db_id': 'perpetrator', 'question': 'Find mean and mean_ad for the perpetrator Injured', 'function': ['numric', ['mean', 'mean_ad'], 'perpetrator', [['Injured'], ['Injured']], [[], []]]}
{'db_id': 'college_2', 'question': 'Provide the mean and var for classroom capacity', 'function': ['numric', ['mean', 'var'], 'classroom', [['capacity'], ['capacity']], [[], []]]}
{'db_id': 'flight_company', 'question': 'What is the mean and median of the flight Velocity', 'function': ['numric', ['mean', 'median'], 'flight', [['Velocity'], ['Velocity']], [[], []]]}
{'db_id': 'icfp_1', 'question': 'Calculate mean and mean for Authorship authOrder', 'function': ['numric', ['mean', 'mean'], 'Authorship', [['authOrder'], ['authOrder']], [[], []]]}
{'db_id': 'body_builder', 'question': 'What is the mean and std for the body_builder Clean_Jerk', 'function': ['numric', ['mean', 'std'], 'body_builder', [['Clean_Jerk'], ['Clean_Jerk']], [[], []]]}
{'db_id': 'storm_record', 'question': 'Find mean and size of th

In [ ]:
## generate paeamerteres naming

import json
with open('/content/train_dev_with_ast.json') as o:
        data = json.load(o)

new_train_and_dev = []
for e in data:
  python = {}
  python['db_id'] = e['db_id']
  python['question'] = e['question']
  python['function'] = {}
  python['function']['main_kind'] = e['function'][0]
  python['function']['sub_kinds'] = e['function'][1]
  python['function']['table'] = e['function'][2]
  python['function']['columns'] = e['function'][3]
  python['function']['extra_param'] = e['function'][4]
  python['python'] = e['python']


  new_train_and_dev.append(python)

In [ ]:
with open('new_train_and_dev.json', 'wt') as data:
   json.dump(new_train_and_dev, data, sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
## create tokenize question
with open('/content/new_train_and_dev.json') as o:
        data = json.load(o)

new_train_and_dev = []
for e in data:
  e['question_tokens'] =  word_tokenize(e['question'])
  new_train_and_dev.append(e)


In [ ]:
with open('new_train_and_dev.json', 'wt') as data:
   json.dump(new_train_and_dev, data, sort_keys=False, indent=4, separators=(',', ': '))

In [ ]:
#saplitted into train and dev datasets
with open('/content/new_train_and_dev.json') as o:
        data = json.load(o)

train = []
dev = []
number = 0
for e in data:
  if e['function']['main_kind'] == 'distribution' and number % 2 == 0 or len(dev) == 50:
    train.append(e)
  elif e['function']['main_kind'] == 'distribution' and number % 2 == 1:
    dev.append(e)
  if e['function']['main_kind'] == 'plot' and number % 2 == 0 or len(dev) == 90:
    train.append(e)
  elif e['function']['main_kind'] == 'plot' and number % 2 == 1:
    dev.append(e)
  if e['function']['main_kind'] == 'query' and number % 2 == 0 or len(dev) == 155:
    train.append(e)
  elif e['function']['main_kind'] == 'query' and number % 2 == 1:
    dev.append(e)
  if e['function']['main_kind'] == 'numric' and number % 2 == 0 or len(dev) == 195:
    train.append(e)
  elif e['function']['main_kind'] == 'numric' and number % 2 == 1:
    dev.append(e)
  number = number + 1

In [ ]:
with open('train.json', 'wt') as data:
   json.dump(train, data, sort_keys=False, indent=4, separators=(',', ': '))

with open('dev.json', 'wt') as data:
   json.dump(dev, data, sort_keys=False, indent=4, separators=(',', ': '))